In [11]:
from __future__ import division

#import torch 
#import torch.nn as nn
#import torch.nn.functional as F 
#from torch.autograd import Variable
import numpy as np

from tensorflow import Variable
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, BatchNormalization, LeakyReLU, UpSampling2D, Concatenate
from tensorflow.keras.layers import Add

In [2]:
def parse_cfg(cfgfile):
    """
    Takes a configuration file
    
    Returns a list of blocks. Each blocks describes a block in the neural
    network to be built. Block is represented as a dictionary in the list
    
    """
    
    file = open(cfgfile, 'r')
    lines = file.read().split('\n')                        # store the lines in a list
    lines = [x for x in lines if len(x) > 0]               # get rid of the empty lines 
    lines = [x for x in lines if x[0] != '#']              # get rid of comments
    lines = [x.strip() for x in lines]           # get rid of fringe whitespaces
    
    block = {}
    blocks = []

    for line in lines:
        if line[0] == "[":               # This marks the start of a new block
            if len(block) != 0:          # If block is not empty, implies it is storing values of previous block.
                blocks.append(block)     # add it the blocks list
                block = {}               # re-init the block
            block["type"] = line[1:-1].rstrip()     
        else:
            key,value = line.split("=") 
            block[key.rstrip()] = value.lstrip()
    blocks.append(block)

    return blocks

def create_modules(blocks):
    net_info = blocks[0]     #Captures the information about the input and pre-processing    
    module_list = Sequential()
    prev_filters = 3
    output_filters = []
    
    outputs = {}
    output_filters = []
    filters = []
    
    for index, x in enumerate(blocks[1:]):
        model = Sequential()

        #check the type of block
        #create a new module for the block
        #append to module_list
        if (x["type"] == "convolutional"):
            #Get the info about the layer
            activation = x["activation"]
            try:
                batch_normalize = int(x["batch_normalize"])
                bias = False
            except:
                batch_normalize = 0
                bias = True

            filters= int(x["filters"])
            padding = int(x["pad"])
            kernel_size = int(x["size"])
            stride = int(x["stride"])

            if padding:
                model.add(ZeroPadding2D(padding=(kernel_size - 1) // 2))

            #Add the convolutional layer
            
            conv = Conv2D(filters, kernel_size, strides=stride, use_bias = bias, padding="valid")
            model.add(conv, name="conv_{0}".format(index))

            #Add the Batch Norm Layer
            if batch_normalize:
                model.add(BatchNormalization(), name="batch_norm_{0}".format(index))

            #Check the activation. 
            #It is either Linear or a Leaky ReLU for YOLO
            if activation == "leaky":
                model.add(LeakyReLU(alpha=0.1), name="leaky_{0}".format(index))

        #If it's an upsampling layer
        #We use Bilinear2dUpsampling
        elif (x["type"] == "upsample"):
            stride = int(x["stride"])
            model.add(UpSampling2D(size=stride, interpolation="bilinear"), name="upsample_{}".format(index))
        
        #If it is a route layer
        elif (x["type"] == "route"):
            
            x["layers"] = x["layers"].split(',')
            
            start = int(x["layers"][0])
            if len(x["layers"]) > 1:
                end = int(x["layers"][1]) - index
                filters = output_filters[index + start] + output_filters[end]  # Index negatif :end - index
                model.add(Concatenate([outputs[index + start], outputs[index + end]], axis=-1), name="route_{}".format(index))
            else:
                filters = output_filters[index + start]
                model.add(Concatenate([outputs[index + start]], axis=-1), name="route_{}".format(index))
                #inputs = outputs[index + start]
        

        #shortcut corresponds to skip connection
        elif x["type"] == "shortcut":
            
            from_ = int(x["from"])
            #inputs = outputs[index - 1] + outputs[index + from_]
            model.add(Add()([outputs[index - 1], outputs[index + from_]]), name="shortcut_{}".format(index))
            
            shortcut = EmptyLayer()
            module.add_module("shortcut_{}".format(index), shortcut)
        
        # Yolo detection layer
        elif x["type"] == "yolo":
            
    
    

In [10]:
blocks = parse_cfg("./cfg/yolov3.cfg")
blocks

[{'type': 'net',
  'batch': '64',
  'subdivisions': '16',
  'width': '608',
  'height': '608',
  'channels': '3',
  'momentum': '0.9',
  'decay': '0.0005',
  'angle': '0',
  'saturation': '1.5',
  'exposure': '1.5',
  'hue': '.1',
  'learning_rate': '0.001',
  'burn_in': '1000',
  'max_batches': '500200',
  'policy': 'steps',
  'steps': '400000,450000',
  'scales': '.1,.1'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '2',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '32',
  'size': '1',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'convolutional',
  'batch_normalize': '1',
  'filters': '64',
  'size': '3',
  'stride': '1',
  'pad': '1',
  'activation': 'leaky'},
 {'type': 'shortcut', 'from': '-3', 'activatio

In [ ]:
https://machinelearningspace.com/yolov3-tensorflow-2-part-2/
    
https://blog.paperspace.com/how-to-implement-a-yolo-v3-object-detector-from-scratch-in-pytorch-part-2/